In [1]:
import duckdb
import pyarrow as pa
import polars as pl
CHUNK_SIZE = 1000000

In [2]:

conn = duckdb.connect("timeball.db", read_only=True)
rel = conn.sql("SELECT * FROM main_models.ml_features")
batch_reader = rel.record_batch(CHUNK_SIZE)
schema = batch_reader.schema

def gen():
    for batch in batch_reader:
        df = pl.from_arrow(batch)
        # Do some stuff in polars, then go back to arrow
        arrow_table = df.to_arrow()
        temp_batcher = arrow_table.to_batches(CHUNK_SIZE)
        for new_batch in temp_batcher:
            yield new_batch

In [3]:
final_batcher = pa.RecordBatchReader.from_batches(schema, gen())


In [4]:
cur = conn.cursor()
cur.execute("CREATE OR REPLACE TEMP TABLE t AS (SELECT * FROM final_batcher)")

batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching
batching


In [7]:
cur.sql("SELECT COUNT(*) FROM t")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     15086910 │
└──────────────┘